In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('full_df.csv', index_col=0)

In [3]:
# get year variable
df['year'] = df['title'].astype(str).str[0:4]
# get year make model variable
df['year_make_model'] = df['year'] + ' ' + df['make'] + ' ' + df['model']
# get clean status
df['title_status'] = df['title.1'].astype(str).str[0:5] == 'Clean'
df['title_status'] = df['title_status'].astype(int)
df['end_bid_new'] = df['end_bid'].str.replace(',', '').str.extract('(\d+)')
df['num_views_new'] = df['num_views'].str.replace(',', '')
df['num_photos_new'] = df['num_photos'].str.extract('(\d+)')
df['milage'] = df['milage'].str.replace(',', '').str.extract('(\d+)')
df['engine'] = df['engine'].str.lower().str.replace(' ', '').str.replace('-','').str.replace('.','')
df['drivetrain'] = df['drivetrain'].str.lower().str.replace(' ', '').str.replace('-', '')
df['transmission'] = df['transmission'].str.lower().str.replace(' ','').str.replace('-','')

In [4]:
working_df = df.drop(columns=['title', 'subtitle', 'VIN', 'location', 'seller', 'num_views', 'end_bid' , 'year', 'make', 'model', 'end_date', 'num_photos', 'title.1', 'e_color', 'i_color'])

In [5]:
trans_dummy = pd.get_dummies(working_df['transmission'])
trans_dummy['manual(5speed)'] = trans_dummy['manual(5speed)'] + trans_dummy['manual(5speedmanual)']
trans_dummy.drop(columns='manual(5speedmanual)', inplace = True)
trans_dummy['automatic(5speed)'] = trans_dummy['automatic(5s[eed)'] + trans_dummy['automatic(5speed)']
trans_dummy.drop(columns='automatic(5s[eed)', inplace = True)

In [6]:
reg_df = working_df.join(trans_dummy)
reg_df.drop(columns='transmission', inplace = True)

In [7]:
working_df.head(3)

,id,reserve,num_bids,num_com,milage,engine,drivetrain,transmission,body_style,seller_type,num_highlights,num_equipment,num_modifications,num_known_flaws,num_service_history,num_other_items,num_owner_history,num_seller_notes,num_videos,sold,sentiment_score,year_make_model,title_status,end_bid_new,num_views_new,num_photos_new
0,1_1,1,16,48,38900,55lturbochargedv8,rearwheeldrive,automatic(7speed),Wagon,Private Party,6,13,16,4,2,10,1,2,2.0,1,0.379978,2012 Mercedes-Benz E63 AMG,1,49500,17729,129
1,1_2,0,15,63,29400,90lturbodieselv6,rearwheeldrive,automatic(4speed),Truck,Private Party,5,10,5,7,0,3,1,2,2.0,1,0.179144,1986 E-One Pumper,1,7877,24166,125
2,1_3,1,19,31,123800,20li6,rearwheeldrive,manual(5speed),Wagon,Private Party,5,10,3,5,10,2,1,2,2.0,1,0.262188,1996 BMW 320i,1,14300,6379,99


In [8]:
model_dummy = pd.get_dummies(working_df['year_make_model'])

In [9]:
reg_df = working_df.join(model_dummy)
reg_df.drop(columns = ['engine', 'drivetrain', 'transmission', 'body_style', 'seller_type', 'year_make_model'], inplace= True)

In [10]:
first_col = reg_df.pop('end_bid_new')

In [11]:
first_col

0        49500
1         7877
2        14300
3        52777
4         8800
         ...  
8400     21000
8401    108000
8402     11000
8403     20750
8404     63000
Name: end_bid_new, Length: 8405, dtype: object

In [12]:
reg_df.insert(0, 'end_bid_new', first_col)

In [13]:
reg_df.drop(columns='id', inplace = True)

In [14]:
reg_df.dropna(inplace = True)

In [15]:
reg_df.dtypes

end_bid_new                    object
reserve                         int64
num_bids                        int64
num_com                         int64
milage                         object
                                ...  
2023 Genesis GV60               uint8
2023 Land Rover Defender        uint8
2023 Land Rover Range Rover     uint8
2023 Nissan Z                   uint8
2023 Toyota Tundra              uint8
Length: 3313, dtype: object

In [16]:
reg_df.apply(pd.to_numeric).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8392 entries, 0 to 8404
Columns: 3313 entries, end_bid_new to 2023 Toyota Tundra
dtypes: float64(2), int32(1), int64(16), uint8(3294)
memory usage: 27.6 MB


In [18]:
reg_df = reg_df.apply(pd.to_numeric)

In [20]:
y = reg_df['end_bid_new']
x = reg_df.iloc[:, 1:]

In [21]:
x = sm.add_constant(x)

In [23]:
model = sm.OLS(y, x)

In [24]:
res = model.fit()

In [29]:
model2 = smf.ols(formula = 'end_bid_new ~ reserve + num_bids + num_com + milage + num_highlights + num_equipment + num_modifications + num_known_flaws + num_service_history + num_other_items + num_videos + sold + sentiment_score + title_status + num_views_new + num_photos_new', data = reg_df).fit()

In [31]:
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:            end_bid_new   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     476.8
Date:                Thu, 19 Jan 2023   Prob (F-statistic):               0.00
Time:                        22:13:46   Log-Likelihood:                -96570.
No. Observations:                8392   AIC:                         1.932e+05
Df Residuals:                    8375   BIC:                         1.933e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.702e+04   3